<h1 style="text-align: center; font-size: 50px;"> 🤖 MLFlow Registration for Multimodal RAG</h1>

In [ ]:
%pip install -r ../requirements.txt --quiet 

# MLFlow Model Service 

In this section, we demonstrate how to deploy a RAG-based chatbot service. This service provides a REST API endpoint that allows users to query the knowledge base with natural language questions, upload new documents to the knowledge base, and manage conversation history, all with built-in safeguards against sensitive information and toxicity. This service encapsulates all the functionality we developed in this notebook, including the document retrieval system, RAG-based question answering capabilities, and Galileo integration for protection, observation and evaluation. It demonstrates how to use our ChatbotService from the src/service directory. 

## Setup

In [ ]:
mlflow_evaluate_setup(
    secrets,
    mlflow_tracking_uri="/phoenix/mlflow"
)

# === Set MLflow experiment context ===
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# === Validate local model file path ===
if not os.path.exists(LOCAL_MODEL_PATH):
    logger.info(f"⚠️ Warning: Model file not found at {LOCAL_MODEL_PATH}. Please verify the path.")

## Log and Register Model

In [ ]:
# === Log and register model to MLflow ===
with mlflow.start_run(run_name=MLFLOW_RUN_NAME) as run:
    
    # Log model artifacts using custom ChatbotService
    ChatbotService.log_model(
        artifact_path=MLFLOW_MODEL_NAME,
        config_path=CONFIG_PATH,
        secrets_path=SECRETS_PATH,
        docs_path=DATA_PATH,
        model_path=LOCAL_MODEL_PATH,
        demo_folder=DEMO_FOLDER
    )

    # Construct the URI for the logged model
    model_uri = f"runs:/{run.info.run_id}/{MLFLOW_MODEL_NAME}"

In [ ]:
# Register the model into MLflow Model Registry
mlflow.register_model(
    model_uri=model_uri,
    name=MLFLOW_MODEL_NAME
)

logger.info(f"✅ Model registered successfully with run ID: {run.info.run_id}")

# Evaluate Hallucinations & Relevance

In [ ]:
model_source = config["model_source"]

In [ ]:
%%time

llm = initialize_llm(model_source, secrets)

In [ ]:

def model(batch_df: pd.DataFrame) -> pd.DataFrame:
    preds, contexts = [], []
    for q in batch_df["questions"]:
        answer = mm_chain.invoke(q)
        preds.append(answer)

        docs = retriever.get_relevant_documents(q)
        contexts.append(" ".join(d.page_content for d in docs))

    # keep the incoming index so every batch’s rows stay unique
    return pd.DataFrame(
        {
            "result": preds,
            "source_documents": contexts,
        },
        index=batch_df.index,      #  ← key line
    )

# --- 3)  Evaluation dataset
eval_df = pd.DataFrame({"questions": [
    "What naming convention should I use for a new blueprint project folder?",
    "What is the first step in the standard blueprint testing workflow?",
    "How do I fetch logs from a running Kubernetes pod?",
]})

judge = LocalGenAIJudge(
    llm=llm
)

faithfulness_metric = judge.to_mlflow_metric("faithfulness")
relevance_metric = judge.to_mlflow_metric("relevance")

results = mlflow.evaluate(
    model,
    eval_df,
    predictions="result",
    evaluators="default",
    extra_metrics=[faithfulness_metric, relevance_metric],
    evaluator_config={
        "col_mapping": {
            "inputs": "questions",
            "context": "source_documents"
        }
    },
)
